In [ ]:
import pandas as pd
import sys
sys.path.append('../utils')
import visualization as visualization
import generic as generic
import project_utils as project_utils
import numpy as np
import matplotlib.pyplot as plt
import importlib
import matplotlib.pyplot as plt

epidemy_years = ['2012_13', '2015_16', '2018_19', '2023_24']


pygame 2.6.1 (SDL 2.28.4, Python 3.11.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Load Data

In [2]:
dengue_data = pd.read_csv('../data/processed/dengue_data_with_coordinates.csv')
daily_ovitraps = pd.read_csv('../data/processed/daily_ovitraps.csv',index_col=0, parse_dates=True)
ovitraps_data = pd.read_csv('../data/processed/ovitraps_data_with_coordinates.csv')


### Process Data

In [3]:
# DELETE WHEN DVC IS READY
daily_dengue_data = project_utils.process_dengue(dengue_data)
# Dengue data
daily_dengue_data = project_utils.get_daily_dengue(dengue_data)

# ovitraps data
daily_ovitraps_sum = daily_ovitraps.sum(axis=1)
daily_ovitraps_mean = daily_ovitraps.mean(axis=1)


In [4]:
# Get the days for each epidemy year  put them in flat list
day_anoepid = project_utils.get_epidemic_years_date_ranges(dengue_data)
epidemy_years_days = {year: days for year, days in day_anoepid.items() if year in epidemy_years}
epidemy_date_range = [
    pd.date_range(start=epidemy_years_days[year][0], end=epidemy_years_days[year][1])
    for year in epidemy_years
]

epidemy_days_flat = [day for period in epidemy_date_range for day in period]

In [5]:
# Add anoepid to daily_dengue
daily_dengue_data_with_anoepid = daily_dengue_data.copy()
daily_dengue_data_with_anoepid = pd.DataFrame(daily_dengue_data_with_anoepid, index = daily_dengue_data_with_anoepid.index)
daily_dengue_data_with_anoepid.rename(columns={0: 'cases'}, inplace=True)

aux_list = [] 
for day in daily_dengue_data_with_anoepid.index:
    for key, value in day_anoepid.items():
        if day in value:
            aux_list.append(key)
daily_dengue_data_with_anoepid['anoepid'] = aux_list

### Define best group for correlation

In [24]:
max_corr = []
max_lag = []
for i in range(1,61):
    offset_days = (daily_ovitraps_mean.index[0] - daily_dengue_data.index[0]  ).days % i
    
    groups_dengue = generic.group_series(
        series = daily_dengue_data,
        offset = offset_days,
        group_size = i,
        operation = "sum"
    )

    groups_ovitraps = generic.group_series(
        series = daily_ovitraps_mean,
        offset = 0,
        group_size = i,
        operation = "mean"
    )

    results = generic.max_correlation(groups_dengue, groups_ovitraps, max_lag = np.ceil(120/i).astype(int))
    
    max_lag.append(results[0])
    max_corr.append(results[1]) 



In [34]:
import plotly.graph_objects as go
go.Figure(data=go.Scatter(x=list(range(1,61)), y=max_corr, mode='markers+lines', name='Max Correlation')) 